# Clickbait Detection: SVM (Scaled) & UMAP Embeddings

## Εισαγωγή
Στο παρόν notebook αναπτύσσουμε ένα μοντέλο **Support Vector Machine (SVM)** με πυρήνα RBF.

### Γιατί Scaling;
Τα μοντέλα SVM που χρησιμοποιούν πυρήνες (kernels) όπως ο RBF βασίζονται στον υπολογισμό Ευκλείδειων αποστάσεων μεταξύ των σημείων.
Εάν τα χαρακτηριστικά δεν είναι στην ίδια κλίμακα, οι διαστάσεις με μεγάλες τιμές/διακυμάνσεις θα κυριαρχήσουν στον υπολογισμό της απόστασης, οδηγώντας σε κακή απόδοση.
Γι' αυτό, η εφαρμογή **`StandardScaler`** είναι υποχρεωτική.

## Στόχοι
1. **Φόρτωση Δεδομένων**: UMAP-500 embeddings.
2. **Βελτιστοποίηση (Optuna)**: Εύρεση των βέλτιστων `C`, `tol` και `class_weight`.
3. **Champion Model**: Εκπαίδευση με `max_iter=15000` για διασφάλιση σύγκλισης.
4. **Αποθήκευση**: Αποθήκευση του Scaler μαζί με το μοντέλο.

In [ ]:
import pandas as pd
import numpy as np
import optuna
import mlflow
import sys
import os
import time
import pickle
import warnings

from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, f1_score
from sklearn.preprocessing import StandardScaler

# Απόκρυψη FutureWarnings
warnings.filterwarnings("ignore", category=FutureWarning, module="sklearn")

# Ρύθμιση Paths
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
import mlflow_helper

project_root = os.path.abspath(os.path.join(os.getcwd(), '../../..'))
DATA_FOLDER = os.path.join(project_root, 'data', 'clean', 'umap')

print(f"Project Root: {project_root}")
print(f"Data Folder: {DATA_FOLDER}")

## Φόρτωση Δεδομένων

Φορτώνουμε τα δεδομένα. Το Scaling θα εφαρμοστεί στο επόμενο βήμα.

In [ ]:
def load_split_data(data_path):
    files = {
        "Train": "train_umap_500.parquet",
        "Valid": "valid_umap_500.parquet",
        "Test":  "test_umap_500.parquet"
    }

    loaded_data = {}
    possible_label_cols = ['labels', 'label', 'target', 'class', 'is_clickbait']

    print(f"⏳ Έναρξη φόρτωσης από: {data_path}")

    for name, filename in files.items():
        file_path = os.path.join(data_path, filename)
        if not os.path.exists(file_path):
            raise FileNotFoundError(f"Το αρχείο {filename} δεν βρέθηκε.")

        try:
            df = pd.read_parquet(file_path, engine='fastparquet')
        except:
            df = pd.read_parquet(file_path, engine='pyarrow')

        # 1. Εντοπισμός Features
        feature_cols = sorted([c for c in df.columns if c.startswith("umap_")])

        # 2. Εντοπισμός Label
        label_col = next((c for c in possible_label_cols if c in df.columns), None)
        if label_col is None:
             raise ValueError(f"Δεν βρέθηκε label στήλη στο {filename}")

        # Διαχωρισμός X, y
        y = df[label_col].values.astype(int)
        X = df[feature_cols].values.astype(np.float32)

        loaded_data[name] = (X, y)
        print(f"   ✅ [{name}] Loaded: X={X.shape}, y={y.shape}")

    return loaded_data["Train"], loaded_data["Valid"], loaded_data["Test"]

# Εκτέλεση Φόρτωσης
(X_train, y_train), (X_val, y_val), (X_test, y_test) = load_split_data(DATA_FOLDER)

## Βελτιστοποίηση Υπερπαραμέτρων (Tuning)

Ορίζουμε τη συνάρτηση `objective`.
* **Kernel**: Χρησιμοποιούμε σταθερά `rbf` με `gamma='auto'`.
* **Scaling**: Εφαρμόζουμε `StandardScaler` εντός του trial (fit στο train, transform στο valid).
* **Max Iterations**: Ορίζουμε `max_iter=15000` γιατί το SVM είναι υπολογιστικά βαρύ και θέλουμε να του δώσουμε περιθώριο σύγκλισης.

In [ ]:
def objective(trial, X_tr, y_tr, X_v, y_v):
    # --- 1. Hyperparameters Space ---
    C = trial.suggest_float("C", 1e-1, 1e2, log=True)
    tol = trial.suggest_float("tol", 1e-5, 5e-4, log=True)
    class_weight = trial.suggest_categorical("class_weight", [None, "balanced"])

    # Σταθερές παράμετροι για SVM
    kernel = "rbf"
    gamma = "auto"
    max_iter = 15000

    # --- 2. Scaling (Critical for SVM) ---
    scaler = StandardScaler()
    X_tr_sc = scaler.fit_transform(X_tr)
    X_v_sc = scaler.transform(X_v)

    # --- 3. Model Setup ---
    params = dict(
        kernel=kernel,
        gamma=gamma,
        C=C,
        tol=tol,
        class_weight=class_weight,
        max_iter=max_iter,
        probability=False, # False για ταχύτητα στο tuning
        random_state=42
    )

    model = SVC(**params)

    # --- 4. Training & Eval ---
    t0 = time.time()
    model.fit(X_tr_sc, y_tr)
    train_time = time.time() - t0

    preds = model.predict(X_v_sc)
    f1 = f1_score(y_v, preds)
    acc = accuracy_score(y_v, preds)

    metrics = {"val_f1": f1, "val_accuracy": acc, "training_time_sec": train_time}

    # Log scaling info
    params_to_log = params.copy()
    params_to_log["scaler"] = "StandardScaler"

    mlflow_helper.log_optuna_trial(trial, params_to_log, metrics, model, run_name_prefix="SVM_Trial")

    return f1

## Εκτέλεση Πειράματος & Champion Model

1. **Tuning:** 10 trials (λόγω του υπολογιστικού κόστους του SVM).
2. **Champion Model:**
   - Επανεκπαίδευση στο Full Dataset (Train + Valid) με **Scaling**.
   - Αποθήκευση του `scaler.pkl` ως artifact.
   - Αξιολόγηση στο Test set (αφού γίνει transform με τον ίδιο scaler).

In [ ]:
EXPERIMENT_NAME = "Clickbait_SVM_UMAP500_Optuna_Scaled"
mlflow_helper.setup_mlflow(EXPERIMENT_NAME)

print(f"\n🚀 Έναρξη Πειράματος: {EXPERIMENT_NAME}")

# --- ΦΑΣΗ 1: Tuning ---
print("\n🔍 [ΦΑΣΗ 1] Hyperparameter Tuning...")
with mlflow.start_run(run_name="SVM_Hyperparameter_Tuning"):
    mlflow.log_param("dataset", "UMAP_500")
    mlflow.log_param("scaler", "StandardScaler")

    study = optuna.create_study(direction="maximize")
    # 10 Trials μόνο, γιατί το SVM είναι αργό
    study.optimize(
        lambda t: objective(t, X_train, y_train, X_val, y_val),
        n_trials=10
    )

    print("🏆 Best F1:", study.best_value)
    print("🏆 Best Params:", study.best_params)


# --- ΦΑΣΗ 2: Champion Model ---
print("\n👑 [ΦΑΣΗ 2] Champion Model Training...")
with mlflow.start_run(run_name="SVM_Champion_Model") as final_run:
    best = study.best_params

    # 1. Προετοιμασία Δεδομένων & Scaling
    X_full = np.vstack([X_train, X_val])
    y_full = np.concatenate([y_train, y_val])

    scaler = StandardScaler()
    X_full_sc = scaler.fit_transform(X_full)
    X_test_sc = scaler.transform(X_test)

    # 2. Setup Model
    champion_params = dict(
        kernel="rbf",
        gamma="auto",
        C=best["C"],
        tol=best["tol"],
        class_weight=best["class_weight"],
        max_iter=15000,
        probability=False,
        random_state=42
    )

    final_model = SVC(**champion_params)

    # 3. Training
    t0 = time.time()
    final_model.fit(X_full_sc, y_full)
    final_train_time = time.time() - t0

    # 4. Logging & Artifacts
    mlflow.log_params(champion_params)
    mlflow.log_param("scaler", "StandardScaler")

    with open("scaler.pkl", "wb") as f:
        pickle.dump(scaler, f)
    mlflow.log_artifact("scaler.pkl")

    mlflow.sklearn.log_model(final_model, artifact_path="champion_model")

    # 5. Evaluation
    print("📈 Αξιολόγηση στο Test Set...")
    mlflow_helper.evaluate_and_log_metrics(
        final_model,
        X_test_sc, # Scaled Data!
        y_test,
        prefix="test",
        training_time=final_train_time
    )

    print(f"\n✅ Ολοκληρώθηκε. Run ID: {final_run.info.run_id}")